In [1]:
import numpy as np
import homework1_5b as hm1b
import homework1_5c as hm1c
import importlib as im

TRAINING_DATA = 'quasar_train.csv'
TESTING_DATA  = 'quasar_test.csv'
RIGHT_FREQ = 1300
LEFT_FREQ  = 1200

In [2]:
def load_csv(file):
    return np.loadtxt(file, delimiter=',')


def load_data(training_data_file=TRAINING_DATA, testing_data_file=TESTING_DATA):
    data_training = load_csv(training_data_file)
    data_testing = load_csv(testing_data_file)
    
    features_training = data_training[0]
    features_testing = data_testing[0]
    
    assert(np.array_equal(features_training, features_testing))
    
    features = features_training
    data_set_tr = data_training[1:]
    data_set_te = data_testing[1:]
    
    return features, data_set_tr, data_set_te


In [3]:
def estimate(tau, features, training):
    w_tau = hm1b.weightM(tau)
    return hm1c.SpectrumModel(training, features, w_tau)


def estimates(taus, features, training):
    models = {}
    for tau in taus:
        models[tau] = estimate(tau, features, training)
        
    return models

In [4]:
tau = 5
features, training, testing = load_data()
mtr = estimate(tau, features, training)
mte = estimate(tau, features, testing)

In [31]:
def d(f1, f2, z):
    diff = f1(z) - f2(z)
    return diff.T.dot(diff)

def d_evaled(fs, f):
    try:
        rows = len(fs)
    except:
        rows = 1
    
    v = np.zeros(len(fs))
    for i in range(rows):
        diff = fs[i] - f
        v[i] = diff.T.dot(diff)
    
    return v

ker = np.vectorize(lambda t: max(1-t, 0))

def right(f, features):
    r = np.where(features == RIGHT_FREQ)
    assert(r)
    return f[:,r[0][0]:]

    
def left(f, features):
    l = np.where(features == RIGHT_FREQ)
    assert(l)
    return f[:,:l[0][0]]


def h(gs, f, features):
    v = np.zeros(len(gs))
    for i in range(len(gs)):
        v[i] = d(gs[i], f, features)
        
    return np.max(v)


def h_evaled(gs, f):
    v = d_evaled(gs, f)
    return np.max(v)


def neighb(k, fs, f_right):    
    assert(k <= len(fs))
    costs = d_evaled(fs, f_right)
    # Get the indices of the k lowest cost functions by index.
    k_indices = costs.argsort()[:k]
    fs[k_indices]
        
    return fs[k_indices]


In [6]:
mtr_eval = mtr(features)

In [7]:
mtr_eval.shape

(200, 450)

In [8]:
he = h_evaled(mtr_eval, mtr_eval[0])

In [27]:
neighb3 = neighb(3)

In [10]:
ker_coefs = ker(d_evaled(mtr_eval, mtr_eval[0])/he)

In [12]:
np.sum(ker_coefs)

135.89154356193382

In [13]:
np.sum(ve/np.sum(ve))

NameError: name 've' is not defined

In [14]:
mtr_eval_left = left(mtr_eval, features)
mtr_eval_right = right(mtr_eval, features)

In [15]:
ker_sum = np.sum(ker_coefs)

In [16]:
ker_sum

135.89154356193382

In [18]:
estimator_vec = ((mtr_eval_left.T * ker_coefs).T)

In [19]:
class Estimator():
    def __init__(self, m_eval, f_right, features):
        self.features = features
        self.m_eval = m_eval
        self.ker_sums = ker_sums
        self.d_max = he
        self.ker_coefs = ker_coefs

In [34]:
neighb(3, mtr_eval, mtr_eval[0]).shape

(3, 450)